In [1]:
import os
from dotenv import find_dotenv, load_dotenv
from pathlib import Path
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

absProjectDir = Path(os.getcwd()).resolve().parents[0]
projectDir = os.path.relpath(absProjectDir,os.curdir)
load_dotenv(find_dotenv())

trainDfPath = os.path.join(projectDir, os.environ.get("REF_PROC_TRAIN_DF"))
testDfPath = os.path.join(projectDir, os.environ.get("REF_PROC_TEST_DF"))
testOrigDir = os.path.join(projectDir, os.environ.get("PROC_TEST_ORIG_DIR"))
testAugmDir = os.path.join(projectDir, os.environ.get("PROC_TEST_AUG_DIR"))
trainOrigDir = os.path.join(projectDir, os.environ.get("PROC_TRAIN_ORIG_DIR"))
trainAugmDir = os.path.join(projectDir, os.environ.get("PROC_TRAIN_AUG_DIR"))
testRootDir = os.path.commonpath([testOrigDir, testAugmDir])
trainRootDir = os.path.commonpath([trainOrigDir, trainAugmDir])

sys.path.append(os.path.join(projectDir,'src/data'))
import imggen as imgen

In [2]:
trgen = imgen.Generator(trainDfPath, trainRootDir, (224,224))
trgen.baseDf.head(5)
trgen.SetDataFrameForGeneration([[0,3],[1,2]],[[32,32],[64,64]])

In [3]:
it = trgen.GetGenerator(bs = 8)
imgs = it.next()

Found 192 validated image filenames belonging to 2 classes.
